In [ ]:
import pandas as pd
import plotly.express as px
import requests
import nltk
from nltk import tokenize
from bs4 import BeautifulSoup

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True


## Uploading dos dados


In [ ]:

df = pd.read_csv('/content/drive/MyDrive/projeto_giovanna_jusbrasil/TCC/timeLogs.csv', on_bad_lines='warn', sep=';',
                 names=['Portal', 'tempo_exec', 'ORG', 'PER', 'TEMP', 'LOC', 'LEG', 'JUR', 'url'], index_col=False)
df

,Portal,tempo_exec,ORG,PER,TEMP,LOC,LEG,JUR,url
0,congresso,8901.6,15,8,4,5,1,0,https://congressoemfoco.uol.com.br/area/congr...
1,congresso,7768.5,30,6,4,4,0,0,https://congressoemfoco.uol.com.br/area/congr...
2,congresso,9354.5,27,21,6,6,0,0,https://congressoemfoco.uol.com.br/area/pais/...
3,congresso,13002.5,46,15,6,30,0,0,https://congressoemfoco.uol.com.br/area/mundo...
4,congresso,7694.5,15,9,4,5,0,0,https://congressoemfoco.uol.com.br/area/justi...
...,...,...,...,...,...,...,...,...,...
446,g1,7060.0,11,22,8,17,0,0,https://g1.globo.com/ba/bahia/noticia/2023/03...
447,g1,15568.5,8,34,6,18,0,0,https://g1.globo.com/mg/sul-de-minas/noticia/...
448,g1,6928.4,15,22,5,15,0,0,https://g1.globo.com/ba/bahia/noticia/2023/03...
449,g1,24533.1,17,22,17,53,0,0,https://g1.globo.com/olha-que-legal/noticia/2...


## Boxplot - portal x tempo

In [ ]:
new_df = df.apply(lambda x: x/1000 if x.name == 'tempo_exec' else x)
new_df

,Portal,tempo_exec,ORG,PER,TEMP,LOC,LEG,JUR,url
0,congresso,8.9016,15,8,4,5,1,0,https://congressoemfoco.uol.com.br/area/congr...
1,congresso,7.7685,30,6,4,4,0,0,https://congressoemfoco.uol.com.br/area/congr...
2,congresso,9.3545,27,21,6,6,0,0,https://congressoemfoco.uol.com.br/area/pais/...
3,congresso,13.0025,46,15,6,30,0,0,https://congressoemfoco.uol.com.br/area/mundo...
4,congresso,7.6945,15,9,4,5,0,0,https://congressoemfoco.uol.com.br/area/justi...
...,...,...,...,...,...,...,...,...,...
446,g1,7.0600,11,22,8,17,0,0,https://g1.globo.com/ba/bahia/noticia/2023/03...
447,g1,15.5685,8,34,6,18,0,0,https://g1.globo.com/mg/sul-de-minas/noticia/...
448,g1,6.9284,15,22,5,15,0,0,https://g1.globo.com/ba/bahia/noticia/2023/03...
449,g1,24.5331,17,22,17,53,0,0,https://g1.globo.com/olha-que-legal/noticia/2...


In [ ]:
fig = px.box(new_df, x="Portal", y="tempo_exec", labels={
                     "Portal": "Portal de notícia",
                     "tempo_exec": "Tempo de execução (s)"})
fig.update_layout(font=dict(size=20))
fig.show()

## Barra Portal x tag

In [ ]:
portais = ['congresso', 'correio', 'jota', 'migalhas', 'g1']
tags = ['ORG', 'PER', 'TEMP', 'LOC', 'LEG', 'JUR']

tag_mean = {'Portal':[], 'Tag': [], 'Média': []}

for i in portais:
  portal_df = df[df['Portal'] == i]
  for tag in tags:
    tag_mean['Portal'].append(i)
    tag_mean['Tag'].append(tag)
    mean = round(portal_df.loc[:, tag].mean(), 2)
    tag_mean['Média'].append(mean)

In [ ]:
df_tag_mean = pd.DataFrame.from_dict(tag_mean)
df_tag_mean

,Portal,Tag,Média
0,congresso,ORG,27.91
1,congresso,PER,13.35
2,congresso,TEMP,7.26
3,congresso,LOC,10.97
4,congresso,LEG,2.25
5,congresso,JUR,0.11
6,correio,ORG,16.69
7,correio,PER,8.27
8,correio,TEMP,4.42
9,correio,LOC,6.96


In [ ]:
fig = px.bar(df_tag_mean, x="Portal", y = 'Média', color="Tag", barmode = 'stack', hover_data=['Média'],)

fig.update_traces(width=.5)
fig.update_layout(font=dict(size=20))
fig.show()


## Dispersão TAM x TEMP

In [ ]:
df

,Portal,tempo_exec,ORG,PER,TEMP,LOC,LEG,JUR,url
0,congresso,8901.6,15,8,4,5,1,0,https://congressoemfoco.uol.com.br/area/congr...
1,congresso,7768.5,30,6,4,4,0,0,https://congressoemfoco.uol.com.br/area/congr...
2,congresso,9354.5,27,21,6,6,0,0,https://congressoemfoco.uol.com.br/area/pais/...
3,congresso,13002.5,46,15,6,30,0,0,https://congressoemfoco.uol.com.br/area/mundo...
4,congresso,7694.5,15,9,4,5,0,0,https://congressoemfoco.uol.com.br/area/justi...
...,...,...,...,...,...,...,...,...,...
446,g1,7060.0,11,22,8,17,0,0,https://g1.globo.com/ba/bahia/noticia/2023/03...
447,g1,15568.5,8,34,6,18,0,0,https://g1.globo.com/mg/sul-de-minas/noticia/...
448,g1,6928.4,15,22,5,15,0,0,https://g1.globo.com/ba/bahia/noticia/2023/03...
449,g1,24533.1,17,22,17,53,0,0,https://g1.globo.com/olha-que-legal/noticia/2...


In [ ]:
df_tam = pd.read_csv('/content/drive/MyDrive/projeto_giovanna_jusbrasil/TCC/pageSize.csv', on_bad_lines='warn', sep=';', header = None)
df_tam.drop(2, inplace=True, axis=1)
df_tam

,0,1
0,https://congressoemfoco.uol.com.br/area/congre...,580
1,https://congressoemfoco.uol.com.br/area/congre...,687
2,https://congressoemfoco.uol.com.br/area/pais/a...,897
3,https://congressoemfoco.uol.com.br/area/mundo-...,1346
4,https://congressoemfoco.uol.com.br/area/justic...,733
...,...,...
446,https://g1.globo.com/ba/bahia/noticia/2023/03/...,909
447,https://g1.globo.com/mg/sul-de-minas/noticia/2...,1722
448,https://g1.globo.com/ba/bahia/noticia/2023/03/...,897
449,https://g1.globo.com/olha-que-legal/noticia/20...,2367


In [ ]:
pages_tam = df_tam[1]


In [ ]:
df = pd.concat([df, pages_tam.rename("pages_tam_tokens")], axis=1)

In [ ]:
df

,Portal,tempo_exec,ORG,PER,TEMP,LOC,LEG,JUR,url,pages_tam_tokens
0,congresso,8901.6,15,8,4,5,1,0,https://congressoemfoco.uol.com.br/area/congr...,580
1,congresso,7768.5,30,6,4,4,0,0,https://congressoemfoco.uol.com.br/area/congr...,687
2,congresso,9354.5,27,21,6,6,0,0,https://congressoemfoco.uol.com.br/area/pais/...,897
3,congresso,13002.5,46,15,6,30,0,0,https://congressoemfoco.uol.com.br/area/mundo...,1346
4,congresso,7694.5,15,9,4,5,0,0,https://congressoemfoco.uol.com.br/area/justi...,733
...,...,...,...,...,...,...,...,...,...,...
446,g1,7060.0,11,22,8,17,0,0,https://g1.globo.com/ba/bahia/noticia/2023/03...,909
447,g1,15568.5,8,34,6,18,0,0,https://g1.globo.com/mg/sul-de-minas/noticia/...,1722
448,g1,6928.4,15,22,5,15,0,0,https://g1.globo.com/ba/bahia/noticia/2023/03...,897
449,g1,24533.1,17,22,17,53,0,0,https://g1.globo.com/olha-que-legal/noticia/2...,2367


In [ ]:
fig = px.scatter(df, x="pages_tam_tokens", y="tempo_exec", color='Portal')
fig.show()